# Simple binary classification

On this notebook we will review some of the techniques and tools we often encounter when working with classification problems. We will give a glimpse on some of the most commonly used models for binary classification in order to use them on the sonar dataset (https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)). We will choose a performance function to study the performance of each model on the data and determine which suits the problem better.

# Reading the data

First we read the dataset and give a labe $v_i, 1\leq i \leq 60$ to each of the $60$ characteristics, and we call the last column the name $t$ for the type of the material (rock or mine).

In [52]:
using CSV, DataFrames
data1 = DataFrame(CSV.read("sonar1.csv", DataFrame))

,v1,v2,v3,v4,v5,v6,v7,v8,v9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.02,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109
2,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337
3,0.0262,0.0582,0.1099,0.1083,0.0974,0.228,0.2431,0.3771,0.5598
4,0.01,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598
5,0.0762,0.0666,0.0481,0.0394,0.059,0.0649,0.1209,0.2467,0.3564
6,0.0286,0.0453,0.0277,0.0174,0.0384,0.099,0.1201,0.1833,0.2105
7,0.0317,0.0956,0.1321,0.1408,0.1674,0.171,0.0731,0.1401,0.2083
8,0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465
9,0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684


In [53]:
#here we replace the categorical labels for numerical ones
data1.t .= replace.(data1.t, "R" => "0");  
data1.t .= replace.(data1.t, "M" => "1");

In [54]:
# here we parse the last column to float number so that the models treat the values as numbers and not strings
data1.t = parse.(Float64, data1.t);

In [108]:
# we can see the data as a matrix and note that the data is of type Float64
M = Matrix(data1[:, 1:60]);

# Dimensionality Reduction (PCA)

We see that it's often better to work with a low dimension characteristic set. For that, we use PCA in order to project into a lower dimension while still keeping around 90% of the information

In [109]:
## Apply PCA to the iris dataset
using MultivariateStats
using Statistics
# center and normalize the data
data = M
data = (data .- mean(data,dims = 1))./ std(data,dims = 1)

208×60 Matrix{Float64}:
 -0.39859    -0.0405504   -0.0268608  …   0.0697021    0.171265  -0.657361
  0.701845    0.420616     1.05308       -0.471269    -0.443484  -0.418842
 -0.128918    0.599621     1.71926        1.30621      0.252153   0.256962
 -0.833544   -0.647348     0.48058       -0.548551    -0.637616   1.03215
  2.04585     0.854476     0.111059      -0.486726     0.446284   0.574988
 -0.0245289   0.208237    -0.419802   …  -0.811309    -0.459662  -0.0610632
  0.110307    1.73433      2.29696        0.981626    -0.702326   0.753877
  0.988915    0.496465     1.05048       -0.502182    -0.508195  -0.239953
 -0.29855    -0.0284145    0.118866       0.208809    -0.330241  -0.856127
 -0.555173   -0.64128     -0.237644      -0.687658    -0.378774  -0.498348
 -1.09887    -0.975018    -0.745085   …  -1.18226     -0.427307  -0.577855
 -0.733505   -0.228658    -0.700847       0.193353    -1.13912   -0.418842
 -0.924885   -0.905237    -0.997504      -0.332162    -0.330241  -0.657361
 

# Detecting outliers
Before we proceed to the dimensionality reduction, we need to identify the characteristics on the observations that are away from the rest because they can affect the training process, that is, we look for the observations that are more than 3 standard deviations away from the mean and consider them as outliers, then we replace them by the mean of the corresponding column(characteristic).

In [110]:
for i = 1:60
    for j = 1:208
        if abs(data[j, i] .> 3)
            data[j, i] = mean(data[:,i])
        end
    end
end
data

208×60 Matrix{Float64}:
 -0.39859    -0.0405504   -0.0268608  …   0.0697021    0.171265  -0.657361
  0.701845    0.420616     1.05308       -0.471269    -0.443484  -0.418842
 -0.128918    0.599621     1.71926        1.30621      0.252153   0.256962
 -0.833544   -0.647348     0.48058       -0.548551    -0.637616   1.03215
  2.04585     0.854476     0.111059      -0.486726     0.446284   0.574988
 -0.0245289   0.208237    -0.419802   …  -0.811309    -0.459662  -0.0610632
  0.110307    1.73433      2.29696        0.981626    -0.702326   0.753877
  0.988915    0.496465     1.05048       -0.502182    -0.508195  -0.239953
 -0.29855    -0.0284145    0.118866       0.208809    -0.330241  -0.856127
 -0.555173   -0.64128     -0.237644      -0.687658    -0.378774  -0.498348
 -1.09887    -0.975018    -0.745085   …  -1.18226     -0.427307  -0.577855
 -0.733505   -0.228658    -0.700847       0.193353    -1.13912   -0.418842
 -0.924885   -0.905237    -0.997504      -0.332162    -0.330241  -0.657361
 

Now we proceed to apply dimensionality reduction on the characteristics dimension. PCA expects the data matrix to be transposed: each row to be a characteristic:

In [111]:
# each obs is now a column, PCA takes features - by - samples matrix
data'

60×208 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.39859     0.701845   -0.128918     …   1.00196     0.0494133   -0.137617
 -0.0405504   0.420616    0.599621         0.159693   -0.0951622   -0.0648223
 -0.0268608   1.05308     1.71926         -0.672222    0.134479    -0.786721
 -0.713384    0.322552    1.16935         -0.530698    0.148463    -0.573683
  0.363579    0.775804    0.399581        -0.721887   -1.05311     -0.968502
 -0.101009    2.60094     2.0883       …   0.211991    0.521607    -1.19736
  0.520383    1.51896     1.96403          0.0639829   0.400618    -0.910318
  0.297126    2.50494     2.8455          -0.199631   -0.264222     0.0610784
  1.12256     1.31515    -3.83207e-16     -0.44095     0.139349     0.0531909
  0.0211349   0.587289    2.12704e-16      0.33211     0.201917     0.201917
 -0.566016    1.92749     2.99377      …   0.268167    0.405314     0.271181
 -0.656956    2.89122    -3.85009e-16     -0.0915326   0.221164    -0.0429861
 -0.351196    2.96

In [112]:
p = fit(PCA,data',maxoutdim=25)

PCA(indim = 60, outdim = 25, principalratio = 0.9067132194194307)

Pattern matrix (unstandardized loadings):
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
            PC1         PC2          PC3           PC4         PC5          PC6          PC7          PC8          PC9         PC10         PC11          PC12          PC13         PC14         PC15         PC16         PC17         PC18          PC19          PC20          PC21          PC22          PC23         PC24         PC25
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

We can see that after getting rid of $35$ columns, we still managed to retain $~90\%$ of the original information.

In [113]:
#projection matrix of the transformation
P = projection(p);

Now we use the transform function to transform the data to the new dimension.

In [180]:
data_transform = MultivariateStats.transform(p, data');

In [182]:
#we transpose again since we had transposed before to apply PCA
data_transform';

In [184]:
#we add the column of labels
A = [data_transform' data1[:,61]];

In [123]:
#and we transform again into a dataframe
df = DataFrame(A, :auto)

,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2.32433,-1.29447,-0.989159,1.40737,0.65821,0.546536,-2.70052
2,-6.21793,-4.50146,-0.0313274,-4.04019,2.93452,-0.685373,0.496122
3,-2.28185,-5.42039,0.591383,-2.05963,2.27768,-2.57987,-0.268366
4,4.0393,-1.81448,-0.80612,1.21011,-2.05167,2.61708,1.52967
5,-1.55175,-0.952711,-0.0203731,-3.58624,2.23203,0.00912788,-0.738891
6,-2.38318,-1.25401,-1.35613,-2.31626,2.80004,1.25014,2.0785
7,-0.901444,-3.16973,-0.905512,0.986358,-1.84549,-1.24892,-1.28336
8,-3.00601,0.183992,-0.466662,0.39736,-0.505127,1.79756,-2.35194
9,-1.19713,0.903343,-0.757028,0.819366,-1.43894,2.47346,-0.0262408


# Splitting the data

The next step is to divide the data into two sets, namely Training and Testing sets; we adjust the models to the training data and then study the performance of the model on the testing set.

In [125]:
using StableRNGs
using MLJ

y, X = unpack(df, ==(:x26)) #split characteristics and labels

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 208×25 DataFrame
 Row │ x1         x2         x3          x4          x5          x6            ⋯
     │ Float64    Float64    Float64     Float64     Float64     Float64       ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  2.32433   -1.29447   -0.989159    1.40737     0.65821     0.546536     ⋯
   2 │ -6.21793   -4.50146   -0.0313274  -4.04019     2.93452    -0.685373
   3 │ -2.28185   -5.42039    0.591383   -2.05963     2.27768    -2.57987
   4 │  4.0393    -1.81448   -0.80612     1.21011    -2.05167     2.61708
   5 │ -1.55175   -0.952711  -0.0203731  -3.58624     2.23203     0.00912788   ⋯
   6 │ -2.38318   -1.25401   -1.35613    -2.31626     2.80004     1.25014
   7 │ -0.901444  -3.16973   -0.905512    0.986358   -1.84549    -1.24892
   8 │ -3.00601    0.183992  -0.466662    0.39736    -0.505127    1.79756
   9 │ -1.19713    0.903343

In [129]:
# Get rocks indices
rocks_idx = findall(==(0),y)
# Get mines indices
mines_idx = findall(==(1),y)

# Set a random seed
rng = StableRNG(0)
# We take 70% of mine indices 
train_mines , test_mines = partition( mines_idx , 0.7, shuffle=true, rng=rng)
# Also 70% of rocks indices 
train_rocks , test_rocks = partition( rocks_idx , 0.7, shuffle=true, rng=rng)

# join mines and rocks (indices) for each set
train = [train_mines ; train_rocks]
test = [test_mines ; test_rocks];

In [141]:
# Take the train set as a dataframe
Train = DataFrame(A[train,:], :auto)

,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.633641,3.2539,1.29587,-2.32477,-0.446616,0.754128,-0.0382978
2,-3.62409,-1.12001,-1.43103,2.00521,0.00899368,0.330671,0.0881543
3,1.56841,0.354458,0.30547,-0.998069,2.36949,0.249138,-0.0492682
4,0.0218187,-3.06259,2.04794,-0.544933,-5.11363,-3.47219,-1.97489
5,-4.51129,-0.222258,1.71048,-0.704169,-0.120704,-1.85756,-1.5639
6,-2.95275,0.648978,-2.24889,0.362719,0.547628,0.73992,3.40731
7,-1.77587,0.267366,0.00617518,3.29647,-3.379,-0.454783,0.288406
8,7.49687,-1.93111,3.86,1.06463,0.73247,0.636394,1.55107
9,-0.080234,2.17526,2.77579,-1.72889,-1.57226,-1.5912,-1.59815


In [143]:
#Take the Test set as a dataframe
Test = DataFrame(A[test,:],:auto)

,x1,x2,x3,x4,x5,x6,x7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2.67422,-2.58367,1.84918,2.94977,2.25763,0.158967,3.19904
2,2.22217,-3.54915,3.27733,-1.42662,-2.76808,-1.50555,0.455601
3,-4.80093,-1.11237,1.64456,0.43256,0.150882,-1.96967,-1.00953
4,-0.9105,3.6121,3.04094,-0.193027,-0.503773,1.19834,0.0913277
5,-2.0373,-1.95942,-1.89185,2.83901,-0.57782,-1.20167,0.610392
6,-3.51161,-2.6956,-1.02456,2.84096,-2.98448,-1.10298,-2.11035
7,-0.195082,2.47236,2.59916,-2.40871,0.706896,-0.892192,-1.2048
8,-0.797876,1.84563,-0.726482,2.81892,0.720546,0.830249,0.397184
9,-4.86489,-4.25104,-1.29747,-4.83094,1.45364,0.540341,0.0524396


# Logistic Regression 
The first classification model we are going to use is Linear Regression. The model basically assigns to each observation a probability of being of one or another type.<br>

For this model we make use of the package GLM (Generalized Linear Models):

In [162]:
using GLM
# We build the model on the train data
fm = @formula(x26 ~ x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12 + x13 + x14 + x15 + x16 + x17 + x18 + x19 + x20 + x21 + x22 + x23 + x24 + x25)
logit = glm(fm, Train, Binomial(), ProbitLink())

# We make the prediction on the test data
prediction = GLM.predict(logit,Test)

# Now we convert the probability that the model assigned to one of the label using 0.5 as the threshold
prediction_class = [if x < 0.5 0 else 1 end for x in prediction]

#here we can contrast the actual data (1st col) with the prediction made by the model(2nd col)
prediction_df = DataFrame(y_actual = Test.x26, y_predicted = prediction_class, prob_predicted = prediction); #model predicted correctly and 0 otherwise

# Performance measure: Recall

But what does the above information tells us about how good the model worked? In order to measure the performance of the model, one way to evaluate the performance is the recall: it tells you the true positive ratio of the labels that we tried to predict, that is
$$
Recall = \frac{TP}{TP+FN}.
$$
 We choose this measure beacuse we want to focus more on detecting true mines rather than worrying too much about false positive calls.

In [164]:
# and finally we measure the performance of the model
logit_recall = recall(prediction_class,Test.x26);
print("Recall in Logistic Regression: $logit_recall")

Recall in Logistic Regression: 0.8484848484848485

┌ Warning: The classes are un-ordered,
│ using: negative='0.0' and positive='1.0'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\jaime\.julia\packages\MLJBase\IVwqP\src\measures\confusion_matrix.jl:116


# Linear Regression

In [167]:
fm = @formula(x26 ~ x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12 + x13 + x14 + x15 + x16 + x17 + x18 + x19 + x20 + x21 + x22 + x23 + x24 + x25)
linearRegressor = lm(fm, Train);
prediction1 = GLM.predict(linearRegressor, Test);

#Clasificación
prediction_class1 = [if x < 0.5 0 else 1 end for x in prediction1];
prediction1 =  prediction_class1;

In [168]:
# and finally we measure the performance of the model
linear_recall = recall(prediction1,Test.x26);
print("Recall in lin reg: $linear_recall")

Recall in lin reg: 0.8181818181818182

┌ Warning: The classes are un-ordered,
│ using: negative='0.0' and positive='1.0'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\jaime\.julia\packages\MLJBase\IVwqP\src\measures\confusion_matrix.jl:116


# SVM

In [179]:
using LIBSVM # SVM library
using RDatasets

# Here we set up the model on the train data
X = Matrix(Train[:, 1:25])' # the X input has to be transposed
y = Train.x26
X0 = Matrix(Test[:, 1:25])'
model = svmtrain(X, y); # svmtrain makes the svm model

# Now we test on the test data
(predicted_labels, decision_values) = svmpredict(model, X0);

# And finally we measure performance
svm_recall = recall(predicted_labels,Test.x26);
print("Recall in svm: $svm_recall")

Recall in svm: 0.9090909090909091

┌ Warning: The classes are un-ordered,
│ using: negative='0.0' and positive='1.0'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\jaime\.julia\packages\MLJBase\IVwqP\src\measures\confusion_matrix.jl:116


# DecisionTree
Next up we have Decision Trees: the model creates a tree which classifies the data by determining which features and with what thresholds split the data into uniform subsets.

In [157]:
using DecisionTree # This is the Julia library for Decision Trees model

Xt = Matrix(Train[:, 1:25]);
yt = Train.x26;

tmodel = DecisionTreeClassifier() #this function creates a tree model to which one can specify or not the desired depht of the tree.

DecisionTree.fit!(tmodel, Xt, yt) # we fit the previous model to the train data
# and we can see the tree, which features it took and the corresponding thresholds
print_tree(tmodel)

Feature 3, Threshold 0.9266514009751665
L-> Feature 9, Threshold -0.8352455675323365
    L-> Feature 2, Threshold 0.7724766883400427
        L-> Feature 22, Threshold 0.6548853999226476
            L-> 1.0 : 15/15
            R-> 0.0 : 2/2
        R-> 0.0 : 5/5
    R-> Feature 1, Threshold -2.4358247013729377
        L-> Feature 19, Threshold -0.1044406986948024
            L-> Feature 10, Threshold -1.9352262271401863
                L-> 1.0 : 1/1
                R-> 0.0 : 7/7
            R-> Feature 2, Threshold 1.2460079662085652
                L-> Feature 5, Threshold 1.1364641583657369
                    L-> 1.0 : 9/9
                    R-> 0.0 : 1/1
                R-> 0.0 : 2/2
        R-> Feature 22, Threshold -0.49042343958770407
            L-> Feature 22, Threshold -0.9938535132870014
                L-> 0.0 : 5/5
                R-> Feature 24, Threshold -0.4217667681550584
                    L-> 0.0 : 2/2
                    R-> 1.0 : 6/6
            R-> Feature 8, Thr

In [161]:
# here we test the model on the test data
Xt0 = Matrix(Test[:,1:25])
yt = DecisionTree.predict(tmodel, Xt0) # the DecisionTree Package has its own predict function

# and finally we measure the performance of the model
tree_recall = recall(yt,Test.x26);
print("Recall in DecisionTree: $tree_recall")

Recall in DecisionTree: 0.6363636363636364

┌ Warning: The classes are un-ordered,
│ using: negative='0.0' and positive='1.0'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\jaime\.julia\packages\MLJBase\IVwqP\src\measures\confusion_matrix.jl:116


# KNN
Finally, we have the K-Nearest Neighbors model for classification: given a point $x$ that we want to classify, what the model does is to take the nearest $k$ neighbors of $x$, and see of which type are most of those neighbors in order to assign to $x$ such label. In order to avoid equal amount of neighbors of different type, one should choose and odd value of $k$. Also $k$ shall not be too big since taking too many neighbors can be counterproductive.<br>

The algorithm itself sounds and is indeed very expensive in terms of computation time, but thanks to the $k$-dimensional tree structure (see https://en.wikipedia.org/wiki/K-d_tree), we can partition the set of point in order to search for neighbors in efficient time.<br>

The NearestNeighbors package of julia implements the concepts mentioned above:

In [175]:
using NearestNeighbors, StatsBase #we add the knn package for julia
# remember  X = Matrix(train[:, 1:60])'
#           y = train.x61
#           X0 = Matrix(test[:, 1:60])'

kdtree = KDTree(X) # create the k-dimensional tree of the data

k = 5 # choose an odd and not too big value for k

index_knn, distances = knn(kdtree, X0, k, true) #this gives the indexes of the respective 5 nearest neighbors and the given distances to each point on the test data X0

# now we put back that information in ma matrix where we can read it

index_knn_matrix = hcat(index_knn...)
# each row are the index of the nearest neighbors to the respective observation of the test dataset
index_knn_matrix_t = permutedims(index_knn_matrix)

62×5 Matrix{Int64}:
  25  107   94   17   83
  63   43   78   56   45
   5   48   52   34   21
  66   30   51   15   37
  11    2  139   71   73
   2   73   11   34    7
  75   18   68   41    9
 119   25  138   92   29
  35   24   20   70   13
  65   27   50    8  103
 130  144   23   59  119
  70  120    5   10  146
  32   43   78   63   60
   ⋮                 
   2   10   93   85   26
 104  109  111  110  130
  55  130   77    1   44
  13   70   35  101   67
 136   84   74   52    5
 110  104  109  111  144
 105  143   82   59    1
  96  118  134  111   90
 104  110  109  111   96
 130  118  126  105  112
  87  102  103   65   47
 108   93  140   71  115

In [185]:
#here we take the classes (labels) of the nearest neighbors to each observation from the test data
knn_classes = y[index_knn_matrix_t]

# now we make the prediction y_hat by taking the classes that appear the most on those 5 neighbors, 
# we do that by counting the classes with countmap function and then with the argmax function from StastBase package:

y_hat = [
    argmax(countmap(knn_classes[i, :], alg = :dict))
    for i in 1:62
];


In [186]:
# and finally we measure the performance of the model
knn_recall = recall(y_hat,Test.x26);
print("Recall in KNN: $knn_recall")

Recall in KNN: 0.7575757575757576

┌ Warning: The classes are un-ordered,
│ using: negative='0.0' and positive='1.0'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\jaime\.julia\packages\MLJBase\IVwqP\src\measures\confusion_matrix.jl:116


# Which model did better?

We can see, based merely on the performance measure of each model, that the one that did better was the SVM. For further studies and in order to determine which of the models work better in a more deep way, we can try hyperparameters tuning and try cross-validation with a subset of the data in order to train a better model.
